In [1]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()

Loading BokehJS ...

In [2]:
import numpy as np
import pandas as pd

In [3]:
from lsst_efd_client import EfdClient, resample

In [4]:
efd_client = EfdClient('summit_efd')

In [5]:
fnames = np.loadtxt("intra_extra_pairs.dat", dtype='S', unpack=True)

In [6]:
fnames = pd.DataFrame.from_csv("data_20200109_match.csv")

/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-4d7b902/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [39]:
fnames = pd.DataFrame.from_csv("missind_data.csv")

/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-4d7b902/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [40]:
fnames.columns

Index(['intra', 'extra'], dtype='object')

In [41]:
for i in fnames.index:
    print(i)

1
2
3
4
5
6
7


In [42]:
out_data = np.zeros(len(fnames), dtype=[('intra', '<U35'), 
                                           ('extra', '<U35'), 
                                           ('az', float), 
                                           ('el', float), 
                                           ('rot_pos', float),
                                           ('x', float), 
                                           ('y', float), 
                                           ('z', float), 
                                           ('u', float), 
                                           ('v', float), 
                                           ('w', float)])

In [43]:
for i in range(len(fnames)):
    print(f"Processing {i+1} of {len(fnames)}: {fnames['intra'][fnames.index[i]]} x {fnames['extra'][fnames.index[i]]} ...")
    fname_intra = str(fnames['intra'][fnames.index[i]]).split(".")[0]
    fname_extra = str(fnames['extra'][fnames.index[i]]).split(".")[0]
    
    my_query_intra = f'SELECT "imageName" FROM "efd"."autogen"."lsst.sal.GenericCamera.logevent_endReadout" where "imageName" = \'{fname_intra}\''
    my_query_extra = f'SELECT "imageName" FROM "efd"."autogen"."lsst.sal.GenericCamera.logevent_endReadout" where "imageName" = \'{fname_extra}\''

    eron_intra = await efd_client.influx_client.query(my_query_intra)
    eron_extra = await efd_client.influx_client.query(my_query_extra)

    t1 = Time(eron_intra.index[0], scale='tai') - TimeDelta(65., format='sec', scale='tai')
    t2 = Time(eron_extra.index[0], scale='tai')

    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], t1, t2)
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth1CalculatedAngle99"], t1, t2)
    hexapod = await efd_client.select_time_series("lsst.sal.ATHexapod.command_moveToPosition",
                                                  ["x", "y", "z", "u", "v", "w"], t1, t2)
    if len(hexapod) < 2:
        print(f"Could not get hexapod position for pair {i+1}: {fname_intra} x {fname_extra}")
        continue

    rot_pos = np.mean(rotator['nasmyth1CalculatedAngle99'])
    el = np.mean(azel['elevationCalculatedAngle99'])
    az = np.mean(azel['azimuthCalculatedAngle99'])
    x = (hexapod['x'][len(hexapod)-2]+hexapod['x'][len(hexapod)-1])/2.
    y = (hexapod['y'][len(hexapod)-2]+hexapod['y'][len(hexapod)-1])/2.
    z = (hexapod['z'][len(hexapod)-2]+hexapod['z'][len(hexapod)-1])/2.
    u = (hexapod['u'][len(hexapod)-2]+hexapod['u'][len(hexapod)-1])/2.
    v = (hexapod['v'][len(hexapod)-2]+hexapod['v'][len(hexapod)-1])/2.
    w = (hexapod['w'][len(hexapod)-2]+hexapod['w'][len(hexapod)-1])/2.
    out_data[i] = (fnames['intra'][fnames.index[i]], fnames['extra'][fnames.index[i]], az, el, rot_pos, x, y, z, u, u, w)
#     print(az, el, rot_pos, x, y, z, u, u, w)
print("done")

Processing 1 of 7: 1578542773-HIP25041-0-1.fits x 1578542801-HIP25041-0-1.fits ...
Processing 2 of 7: 1578543015-HIP23043-0-1.fits x 1578543043-HIP23043-0-1.fits ...
Processing 3 of 7: 1578544378-HIP23043-0-1.fits x 1578544406-HIP23043-0-1.fits ...
Processing 4 of 7: 1578545134-HIP23043-0-1.fits x 1578545162-HIP23043-0-1.fits ...
Processing 5 of 7: 1578545384-HIP23043-0-1.fits x 1578545412-HIP23043-0-1.fits ...
Processing 6 of 7: 1578545714-HIP23043-0-1.fits x 1578545742-HIP23043-0-1.fits ...
Processing 7 of 7: 1578545943-HIP23043-0-1.fits x 1578545971-HIP23043-0-1.fits ...
done


In [44]:
out_data

array([('1578542773-HIP25041-0-1.fits', '1578542801-HIP25041-0-1.fits', -27.40144044, 47.61859831, -11.36134086, -4.34999921, -1.33812000e-06, 0.46500156, 0.35000065, 0.35000065, 0.),
       ('1578543015-HIP23043-0-1.fits', '1578543043-HIP23043-0-1.fits', -30.71004185, 36.76567418,  -7.61457508, -4.34999921, -1.33812000e-06, 0.46500156, 0.35000065, 0.35000065, 0.),
       ('1578544378-HIP23043-0-1.fits', '1578544406-HIP23043-0-1.fits', -36.51394798, 34.03344129,  -2.57646139, -4.34999921, -1.33812000e-06, 0.46500156, 0.35000065, 0.35000065, 0.),
       ('1578545134-HIP23043-0-1.fits', '1578545162-HIP23043-0-1.fits', -39.51416606, 32.35094303,  -0.34374292, -4.34999921, -1.33812000e-06, 0.50000156, 0.35000065, 0.35000065, 0.),
       ('1578545384-HIP23043-0-1.fits', '1578545412-HIP23043-0-1.fits', -40.47001164, 31.77280513,  -0.10509708, -4.34999921,  2.99998662e-01, 0.50000156, 0.35000065, 0.35000065, 0.),
       ('1578545714-HIP23043-0-1.fits', '1578545742-HIP23043-0-1.fits', -41.7084

In [45]:
df = pd.DataFrame(out_data)

In [46]:
df.to_csv("missing_20200109_match.csv")

In [ ]:
np.mean(df1['elevationCalculatedAngle0'])

In [ ]:
len("1578531387-HIP25531_intra-0-1.fits")

In [ ]:
np.sqrt(4/1.5)

In [ ]:
offset = {'m1': 0.0,
                       'm2': 0.0,
                       'x': 0.0,
                       'y': 0.0,
                       'z': 0.0,
                       'u': 0.0,
                       'v': 0.0
                       }

In [ ]:
print(f"{offset!s}")

In [26]:
out_data['']

array([('1578531387-HIP25531_intra-0-1.fits', '1578531410-HIP25531_extra-0-1.fits',  78.53435196, 60.92252097, -16.39188424, -4.3       ,  1.20000000e+00, 0.5       , 0.35      , 0.35      , 0.),
       ('1578531549-HIP25531-0-1.fits', '1578531579-HIP25531-0-1.fits',  77.94976867, 61.52488464, -15.53119198, -4.29999921,  1.19999866e+00, 0.50000156, 0.35000065, 0.35000065, 0.),
       ('1578532646-HIP25531-0-1.fits', '1578532669-HIP25531-0-1.fits',  73.75123271, 65.33968396,  -9.58440603, -4.29999921,  1.19999866e+00, 0.56000156, 0.35000065, 0.35000065, 0.),
       ('1578532897-HIP25531-0-1.fits', '1578532920-HIP25531-0-1.fits',  72.64797333, 66.20606561,  -8.08566468, -4.29999921,  1.19999866e+00, 0.56800156, 0.35000065, 0.35000065, 0.),
       ('1578533133-HIP25531-0-1.fits', '1578533156-HIP25531-0-1.fits',  71.54836022, 67.01914682,  -6.61612181, -4.29999921,  1.19999866e+00, 0.56800156, 0.35000065, 0.35000065, 0.),
       ('1578533524-HIP25531-0-1.fits', '1578533547-HIP25531-0-1.fit

In [48]:
val = (1,2,3,4,5,6)

In [49]:
names = 'xyzuvw'

In [51]:
dict(zip(names, val))

{'x': 1, 'y': 2, 'z': 3, 'u': 4, 'v': 5, 'w': 6}